###### Content under Creative Commons Attribution license CC-BY 4.0, code under BSD 3-Clause License © 2018  by D. Koehn, notebook style sheet by L.A. Barba, N.C. Clementi

In [1]:
# Execute this cell to load the notebook's style sheet, then ignore it
from IPython.core.display import HTML
css_file = '../style/custom.css'
HTML(open(css_file, "r").read())

# Solving the isotropic elastic 2D SH problem by the staggered grid FD method

Modelling wave propagation in the solid earth by the acoustic wave equation is a very rough approximation. It is not able to describe S-body waves or Rayleigh/Love surface waves and therefore also not any PS or SP conversions. 

The most simple elastic approximation is the 2D SH problem, which can be described by partial differential equations, which are mathematically equal to the 2D acoustic problem. Let's begin with a short review of the 2D SH problem.

## Equations of motion for 2D SH wave propagation in an isotropic elastic medium

As described in [this lesson](http://nbviewer.jupyter.org/github/daniel-koehn/Theory-of-seismic-waves-II/blob/master/01_Analytical_solutions/4_2D_1D_elastic_acoustic_approx.ipynb), we can decompose the 3D isotropic elastic problem into two decoupled problems, the 2D PSV and 2D SH problem. Assuming only non-zero particle displacements in the y-direction perpendicular to the x-z-plane leads to the **2D SH problem**, where x denotes the horizontal distance and z the depth, wave propagation can be described by the following system of partial differential equations:

\begin{align}
\rho \frac{\partial^2 u_y}{\partial t^2} &= \frac{\partial \sigma_{yx}}{\partial x} + \frac{\partial \sigma_{yz}}{\partial z} + f_y, \notag\\
\sigma_{yx} &= \mu \frac{\partial u_{y}}{\partial x},\\
\sigma_{yz} &= \mu \frac{\partial u_{y}}{\partial z}, \notag\\
\end{align}

where $\rho$ is the density and $\mu$ the shear modulus, $\rm{u_y}$ the particle displacement in y-direction, $\sigma_{yx}$, $\rm{\sigma_{yz}}$ shear stress components, $f_y$ the directed body force in y-direction, respectively. 

We can modify this problem by introducing the **particle velocity** $v_y = \frac{\partial u_y}{\partial t}$. Taking the first temporal derivative of the stress-strain relation in eq.(1) and assuming that the shear modulus $\mu$, yields a system of coupled first order partial differential equations:

\begin{align}
\rho \frac{\partial v_y}{\partial t} &= \frac{\partial \sigma_{yx}}{\partial x} + \frac{\partial \sigma_{yz}}{\partial z} + f_y, \notag\\
\frac{\partial \sigma_{yx}}{\partial t} &= \mu \frac{\partial v_{y}}{\partial x},\\
\frac{\partial \sigma_{yz}}{\partial t} &= \mu \frac{\partial v_{y}}{\partial z}. \notag\\
\end{align}

This is known as the **stress-velocity formulation** of the 2D SH problem. Next, we solve eqs. (2) by a staggered grid finite difference approach.

## Solving the 2D SH problem by the Finite difference method on a staggered grid

As for the previous FD approximation of the 1D and 2D acoustic wave equation, we discretize the elastic equations of motion in time and space on a Cartesian grid. However, compared to the acoustic wave equation, the 2D SH problem eqs. (2) involves three wavefields $v_y,\; \sigma_{yx},\; \sigma_{yz}$ and two material parameter $\rho,\; \mu$ distributions. 

In order to evaluate the spatial derivatives correctly in relation to the wavefields and material parameters, [Jean Virieux](https://jean-virieux.obs.ujf-grenoble.fr/) introduced in [this paper](http://jean-virieux.obs.ujf-grenoble.fr/IMG/pdf/GPY_1984_VIRIEUX.pdf) the following **Staggered Finite Difference grid** for the 2D SH problem:

<img src="images/SG_SH-Cart.png" width="50%">

The particle velocity $v_y$ and density $\rho$ are located at the grid point (i,j), while the shear stress $\sigma_{yx}$ are displaced by half a grid point in x-direction (i+1/2,j) and shear stress $\sigma_{yz}$ are displaced by half a grid point in z-direction (i,j+1/2). Furthermore, the shear moduli have to be averaged harmonically:

\begin{align}
\mu_{x} &= \mu(i+1/2,j) = \frac{1}{2}\biggl[\biggl(\mu^{-1}_{i,j}+\mu^{-1}_{i+1,j}\biggr)\biggr]^{-1} \notag\\
\mu_{z} &= \mu(i,j+1/2) = \frac{1}{2}\biggl[\biggl(\mu^{-1}_{i,j}+\mu^{-1}_{i,j+1}\biggr)\biggr]^{-1} \notag
\end{align}

$dx,\; dz$ denotes the spatial distance between two adjacent spatial grid points in x- and z-direction, respectively. The next question is how to approximate the spatial partial derivatives on this staggered grid. 

We begin with the momentum equation

\begin{equation}
\rho \frac{\partial v_y}{\partial t} = \frac{\partial \sigma_{yx}}{\partial x} + \frac{\partial \sigma_{yz}}{\partial z} + f_y. \notag
\end{equation}

The approach can be split into 2 steps:

1. Where are the material parameters and wavefields on the LHS of the equation are located on the staggered grid? In case of the momentum equation they are at (i,j).
<img src="images/SG_SH-Cart.png" width="50%">
2. All derivatives on the RHS have to be centered on the grid point estimated in step 1. Therefore, the spatial derivatives on the RHS of the momentum equation are approximated by  2nd order centered finite-difference operators:

\begin{align}
\biggl(\frac{\partial \sigma_{yx}}{\partial x}\biggr)^c(i,j) &\approx \frac{\sigma_{yx}(i+1/2,j) - \sigma_{yx}(i-1/2,j)}{dx},\notag\\ 
\biggl(\frac{\partial \sigma_{yz}}{\partial z}\biggr)^c(i,j) &\approx \frac{\sigma_{yz}(i,j+1/2) - \sigma_{yz}(i,j-1/2)}{dz}.\notag
\end{align}

For the stress-strain relation of shear stress $\sigma_{yx}$

\begin{equation}
\frac{\partial \sigma_{yx}}{\partial t} = \mu \frac{\partial v_{y}}{\partial x}, \notag
\end{equation}

we use the same approach:

1. $\sigma_{yx}$ is located at the grid point (i+1/2,j)
<img src="images/SG_SH-Cart.png" width="50%">
2. Therefore, we approximate the spatial derivative on the RHS by the 2nd order centered operator:

\begin{equation}
\mu_x \biggl(\frac{\partial v_{y}}{\partial x}\biggr)^c(i+1/2,j) \approx \mu_x \frac{v_y(i+1,j) - v_{y}(i,j)}{dx},\notag 
\end{equation}

For the stress-strain relation of shear stress $\sigma_{yz}$

\begin{equation}
\frac{\partial \sigma_{yz}}{\partial t} = \mu \frac{\partial v_{y}}{\partial z}, \notag
\end{equation}

we get:

1. $\sigma_{yz}$ is located at the grid point (i,j+1/2)
<img src="images/SG_SH-Cart.png" width="50%">
2. Therefore, we approximate the spatial derivative on the RHS by the 2nd order centered operator:

\begin{equation}
\mu_z \biggl(\frac{\partial v_{y}}{\partial x}\biggr)^c(i,j+1/2) \approx \mu_z \frac{v_y(i,j+1) - v_{y}(i,j)}{dz}.\notag 
\end{equation}


So in summary, we get the following spatial FD approximations of the 2D isotropic SH problem:

\begin{align}
\rho \frac{\partial v_y}{\partial t} (i,j)&= \biggl(\frac{\partial \sigma_{yx}}{\partial x}\biggr)^c(i,j) + \biggl(\frac{\partial \sigma_{yz}}{\partial z}\biggl)^c(i,j) + f_y(i,j), \notag\\
\frac{\partial \sigma_{yx}}{\partial t} (i+1/2,j) &= \mu_x \biggl(\frac{\partial v_{y}}{\partial x}\biggr)^c(i+1/2,j),\notag\\
\frac{\partial \sigma_{yz}}{\partial t}(i,j+1/2) &= \mu_z \biggl(\frac{\partial v_{y}}{\partial z}\biggr)^c(i,j+1/2). \notag\\
\end{align}

Finally, we approximate the time derivatives on the LHS of the partial differential equations using the **Leapfrog method** which is also based on a staggered temporal grid at time steps n and n+1/2:

\begin{align}
\scriptsize\rho \frac{v_y(i,j,n+1/2)-v_y(i,j,n-1/2)}{dt}&\scriptsize= \biggl(\frac{\partial \sigma_{yx}}{\partial x}\biggr)^c(i,j,n) + \biggl(\frac{\partial \sigma_{yz}}{\partial z}\biggl)^c(i,j,n) + f_y(i,j,n), \notag\\
\scriptsize\frac{\sigma_{yx}(i+1/2,j,n+1)-\sigma_{yx}(i+1/2,j,n)}{dt} &\scriptsize= \mu_x \biggl(\frac{\partial v_{y}}{\partial x}\biggr)^c(i+1/2,j,n+1/2),\notag\\
\scriptsize\frac{\sigma_{yz}(i,j+1/2,n+1)-\sigma_{yz}(i,j+1/2,n)}{dt} &\scriptsize= \mu_z \biggl(\frac{\partial v_{y}}{\partial z}\biggr)^c(i,j+1/2,n+1/2). \notag\\
\end{align}

Rearranging to the time steps n+1/2 and n+1 on the LHS, yields the 2D SH FD scheme on the staggered grid:

\begin{align}
\scriptsize 
v_y(i,j,n+1/2) &\scriptsize= \frac{dt}{\rho}\biggl\{\biggl(\frac{\partial \sigma_{yx}}{\partial x}\biggr)^c(i,j,n) + \biggl(\frac{\partial \sigma_{yz}}{\partial z}\biggl)^c(i,j,n) + f_y(i,j,n)\biggr\} + v_y(i,j,n-1/2), \notag\\
\scriptsize\sigma_{yx}(i+1/2,j,n+1) &\scriptsize= dt\; \mu_x \biggl(\frac{\partial v_{y}}{\partial x}\biggr)^c(i+1/2,j,n+1/2) + \sigma_{yx}(i+1/2,j,n),\notag\\
\scriptsize\sigma_{yz}(i,j+1/2,n+1) &\scriptsize= dt\; \mu_z \biggl(\frac{\partial v_{y}}{\partial z}\biggr)^c(i,j+1/2,n+1/2) + \sigma_{yz}(i,j+1/2,n). \notag\\
\end{align}


## What we learned:

- How to solve the 2D SH problem by the staggered grid finite difference approach